In [21]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pyro import clear_param_store
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import arviz as az
from scipy.optimize import curve_fit
#from print_versions import print_versions
from seaborn import clustermap

In [22]:
train=pd.read_excel(r"../data/chem_train_pm.xlsx")
test=pd.read_excel(r"../data/chem_test_pm.xlsx")

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
x=torch.tensor(train.corrected_week).float().to(device)
y_pm=torch.tensor(train.pm25_Gaussion).float().to(device)
y_bc=torch.tensor(train.BC_Gaussion).float().to(device)

## PM2.5

In [25]:
clear_param_store()
rbf = gp.kernels.RBF(input_dim=1)


rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(20.)))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(20.)))
gpr = gp.models.GPRegression(x,y_pm, rbf).to(device)
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(40.)))


nuts_kernel = NUTS(gpr.model)


mcmc = MCMC(nuts_kernel,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [06:36, 25.20it/s, step size=8.28e-01, acc. prob=0.897]


In [26]:
torch.save(gpr, "../models/pm25_seasonality");

In [27]:
posterior_samples = mcmc.get_samples(500)
posterior_predictive= Predictive(gpr, posterior_samples)(x)
prior = Predictive(gpr, num_samples=500)(x)

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,

)
az.to_json(pyro_data, "../Arviz_stats/mcmc_pm25.json")

/data/michaelf/miniconda3/lib/python3.10/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_pm25.json'

In [28]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)


rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(20.)))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(20.)))
gpr_bc = gp.models.GPRegression(x,y_bc, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(3.)))


nuts_kernel_bc = NUTS(gpr_bc.model)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc_bc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [10:06, 16.48it/s, step size=5.27e-01, acc. prob=0.917]


In [29]:
torch.save(gpr_bc, "../models/seasonality_BC");

In [30]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(x)
prior_bc = Predictive(gpr_bc, num_samples=500)(x)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_BC.json")

'../Arviz_stats/mcmc_BC.json'

In [31]:
az.summary(pyro_data_bc),az.summary(pyro_data)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)
arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


(                      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
 kernel.lengthscale  25.367  4.168  16.825   32.752      0.166    0.147   
 kernel.variance      2.110  3.275   0.174    6.255      0.146    0.543   
 noise                0.305  0.011   0.284    0.324      0.000    0.000   
 
                     ess_bulk  ess_tail  r_hat  
 kernel.lengthscale     627.0     541.0    NaN  
 kernel.variance        716.0     706.0    NaN  
 noise                  641.0     643.0    NaN  ,
                        mean      sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
 kernel.lengthscale   23.430   4.171   14.897   30.728      0.158    0.133   
 kernel.variance      58.203  10.568   41.153   79.987      0.335    0.315   
 noise               391.928  11.829  370.449  413.953      0.335    0.393   
 
                     ess_bulk  ess_tail  r_hat  
 kernel.lengthscale     672.0     533.0    NaN  
 kernel.variance        981.0     741.0    NaN  
 noise                 1241.0     717.0 

In [32]:
test_1=gpr_bc(torch.tensor(test.corrected_week.values).float().to(device))[0].cpu().detach().numpy()
test_2=gpr(torch.tensor(test.corrected_week.values).float().to(device))[0].cpu().detach().numpy()